In [1]:
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow.keras import backend as K

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import jovian

import os
os.environ["TF_KERAS"] = "1" # for radam env
import gc

import sys
sys.path.append('..')
from harang import vision, utils

import segmentation_models as sm
sm.set_framework('tf.keras')
from efficientnet.tfkeras import preprocess_input

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from keras_radam import RAdam

import efficientnet.tfkeras as eff
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model

from sklearn.metrics import precision_recall_curve, auc

c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' 

<IPython.core.display.Javascript object>

W1118 08:43:55.269525 19228 deprecation_wrapper.py:119] From ..\harang\vision.py:22: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1118 08:43:55.270524 19228 deprecation_wrapper.py:119] From ..\harang\vision.py:24: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1118 08:43:55.301531 19228 deprecation_wrapper.py:119] From ..\harang\vision.py:25: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.



Segmentation Models: using `keras` framework.


Using TensorFlow backend.


In [2]:
target_size = (350, 525)
img_size = (320, 480)
lr = 0.0003
augments = {
    'hf': {'p': 0.5},
    'vf': {'p': 0.5},
    'hsv': {'hue_shift_limit': 5, 'sat_shift_limit': 5, 'val_shift_limit': 5, 'p': 0.5},
    'ssr': {'shift_limit': 0.1, 'scale_limit': 0, 'rotate_limit': 180, 'border_mode': 0, 'value': (0,0,0), 'p': 0.5},
    'bc': {'brightness_limit': 0.1, 'contrast_limit': 0.1, 'p': 0.5},
    'rgb_shift': {'r_shift_limit': 5, 'g_shift_limit': 5, 'b_shift_limit': 5, 'p': 0.5},
    'gamma': {'gamma_limit': (70, 130), 'p': 0.5},
}

pseudo = 'stage1'
stage = 'stage2'

In [3]:
def get_data():
    x_data = utils.from_pickle(f'data/x_data.pkl')
    y_data = utils.from_pickle(f'data/y_data.pkl')
    test = utils.from_pickle(f'data/test.pkl')
    sub = pd.read_csv('data/sample_submission.csv')
    folds = utils.from_pickle('data/folds.pkl')
    if pseudo is None:
        return x_data, y_data, folds, test, sub
    x_pseudo = utils.from_pickle(f'{pseudo}/pseudo/seg/x_pseudo.pkl')
    y_pseudo = utils.from_pickle(f'{pseudo}/pseudo/seg/y_pseudo.pkl')
    return x_data, y_data, x_pseudo, y_pseudo, folds, test, sub

In [4]:
def set_seed(seed):
    np.random.seed(seed)
    rn.seed(seed)
    tf.set_random_seed(seed)

In [5]:
def single_dice_metric(y_true, y_pred_bin):
    intersection = np.sum(y_true * y_pred_bin)
    if (np.sum(y_true)==0) and (np.sum(y_pred_bin)==0):
        return 1
    return (2*intersection) / (np.sum(y_true) + np.sum(y_pred_bin))

def dice_metric(y_true, y_pred_bin):
    batch_size = y_true.shape[0]
    channel_num = y_true.shape[-1]
    mean_dice_metric = 0.
    for i in range(batch_size):
        for j in range(channel_num):
            mean_dice_metric += single_dice_metric(y_true[i, :, :, j], y_pred_bin[i, :, :, j]) / (channel_num*batch_size)
    return mean_dice_metric

def metric_fn(model, x_val, y_val, preprocess_input):
    y_pred = model.predict(preprocess_input(x_val.copy()), batch_size=batch_size*4)[0]
    return dice_metric(y_val, y_pred>0.5)

In [6]:
if pseudo is None:
    x_data, y_data, folds, test, sub = get_data()
else:
    x_data, y_data, x_pseudo, y_pseudo, folds, test, sub = get_data()
    
print(x_data.shape, y_data.shape, x_pseudo.shape, y_pseudo.shape, test.shape)

(5546, 320, 480, 3) (5546, 320, 480, 4) (13686, 320, 480, 3) (13686, 320, 480, 4) (3698, 320, 480, 3)


In [7]:
refs = [
    (sm.Unet, 'efficientnetb2', 8),
    (sm.Unet, 'efficientnetb3', 6),
    (sm.FPN, 'efficientnetb2', 6),
]

In [8]:
gc.collect();
resume_from = 10
resume_count = 0

for m, arch_name, batch_size in refs:
    
    print(f'ARCHITECTURE: {arch_name.upper()}')
        
    if resume_count >= resume_from:
        oof_pred = np.zeros(y_data.shape, dtype=np.float32)
        test_pred = np.zeros((len(test),*img_size,4), dtype=np.float32)
    
    for i in range(5):
        
        resume_count += 1
        if resume_count <= resume_from:
            continue
        
        print(f'FOLD: {i}')
        
        fold = folds[i]
        x_train, x_val, y_train, y_val = x_data[fold[0]], x_data[fold[1]], y_data[fold[0]], y_data[fold[1]]
        
        if pseudo:
            x_train = np.concatenate([x_train, x_pseudo])
            y_train = np.concatenate([y_train, y_pseudo])
            cls_dropout = 0.5
            drop_connect_rate = 0.3
            stochastic_depth = (0.8, 'linear_decay')
            
        else:
            cls_dropout = 0
            drop_connect_rate = 0.2
            stochastic_depth = None
            
        set_seed(i)
        
        model = m(
            backbone_name=arch_name,
            encoder_weights=f'{stage}/cls_weights/{arch_name}_{i}.h5',
            input_shape=(None,None,3),
            classes=4,
            activation='sigmoid',
            cls_mul=True,
            cls_dropout=cls_dropout,
            stochastic_depth=None, 
            drop_connect_rate=drop_connect_rate
        )
        model.compile(
            optimizer=RAdam(lr=lr),
            loss= [sm.losses.BinaryCELoss(), 'binary_crossentropy'],
            loss_weights=[1,0]
        )
        
        train_generator = vision.Generator(
            x_train, 
            y_train,
            batch_size=batch_size,
            augment='both',
            preprocess_input=preprocess_input,
            cls_y=True,
            soft_mask=True,
            **augments
        )
        
        cb = vision.KerasCallback(
            metric_fn=lambda m: metric_fn(m, x_val, y_val, preprocess_input),
            rp=True,
            rp_patience=1,
            rp_factor=0.2,
            decay_factor=1,
            lr=lr,
            patience=3, 
        )
        
        history = model.fit_generator(
            train_generator,
            epochs=1000,
            verbose=1,
            callbacks=[cb],
        )
        
        
        print(cb.best_score)
        print(cb.lr_schedule)       

        save_name = f'{arch_name}_{i}_{str(cb.best_score)[:6]}'
        model.save(f'{stage}/models/seg/{save_name}.h5')
        
        val_pred = np.zeros(y_val.shape, dtype=np.float32)
        tmp_test_pred = np.zeros((len(test),*img_size,4), dtype=np.float32)
        for hf in [{'p': 1.0}, False]:
            for vf in [{'p': 1.0}, False]:
                for case, data in [('x_val', x_val), ('test', test)]:
                    tta_generator = vision.Generator(
                        data,
                        batch_size=batch_size*4,
                        augment='image',
                        hf=hf,
                        vf=vf,
                        preprocess_input=preprocess_input
                    )
                    single_pred = model.predict_generator(tta_generator, verbose=1)[0]
                    if hf:
                        single_pred = np.flip(single_pred, axis=2)
                    if vf:
                        single_pred = np.flip(single_pred, axis=1)
                    if case == 'x_val':
                        val_pred += single_pred/4
                    elif case == 'test':
                        tmp_test_pred += single_pred/4
        
                
        utils.to_pickle(f'{stage}/oof_preds/seg/{save_name}.pkl', val_pred)
        utils.to_pickle(f'{stage}/test_preds/seg/{save_name}.pkl', tmp_test_pred)
                
        oof_pred[fold[1]] = val_pred
        test_pred += tmp_test_pred/5
        
        K.clear_session()
        gc.collect();
        del model
        gc.collect();
        
        jovian.commit(nb_filename='segmentation.ipynb', secret=True, env_type='pip')
    
    if resume_count > resume_from:
        score_str = str(dice_metric(y_data, oof_pred>0.5))[:6]
        print(f'OOF SCORE: {score_str}')
        utils.to_pickle(f'{stage}/oof_preds/seg/{arch_name}_{score_str}.pkl', oof_pred)
        utils.to_pickle(f'{stage}/test_preds/seg/{arch_name}_{score_str}.pkl', test_pred)
        
        jovian.commit(nb_filename='segmentation.ipynb', secret=True, env_type='pip')

ARCHITECTURE: EFFICIENTNETB2
ARCHITECTURE: EFFICIENTNETB3
ARCHITECTURE: EFFICIENTNETB2
FOLD: 0


W1118 08:45:56.278898 19228 deprecation.py:573] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\util\deprecation.py:507: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with distribution=normal is deprecated and will be removed in a future version.
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
W1118 08:45:56.414929 19228 deprecation.py:506] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1118 08:46:05.641016 19228 deprecation.py:323] From c:\users\vnfma\gpu\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future

Epoch 1/1000
Score: 0.6374 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1550
3021/3021 [==============================] - 1549s 513ms/step - loss: 0.1938 - multiply_loss: 0.1938 - cls_output_loss: 0.3699
Epoch 2/1000
Score: 0.6443 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1485
3021/3021 [==============================] - 1486s 492ms/step - loss: 0.1734 - multiply_loss: 0.1734 - cls_output_loss: 0.3090
Epoch 3/1000
Score: 0.6422 | LR to: 0.0003000000142492354                                                                                                     
Restoring Best Weights
Reducing LR on Plateau, Setting LR to 6.000000284984708e-05
TIME SPENT: 1489
3021/3021 [==============================] - 1489s 493ms/step - loss: 0.1683 - multiply_loss: 0.1683 - cls_output_loss: 0.2976
Epoch 4/

<IPython.core.display.Javascript object>

[jovian] Updating notebook "b00034290bee47dab68436fc6b62fbd8" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/harangdev/b00034290bee47dab68436fc6b62fbd8
FOLD: 1
Epoch 1/1000
Score: 0.6343 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1558
3021/3021 [==============================] - 1557s 515ms/step - loss: 0.1918 - multiply_loss: 0.1918 - cls_output_loss: 0.3531
Epoch 2/1000
Score: 0.6429 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1504
3021/3021 [==============================] - 1505s 498ms/step - loss: 0.1715 - multiply_loss: 0.1715 - cls_output_loss: 0.2996
Epoch 3/1000
Score: 0.6426 | LR to: 0.0003000000142492354                                                                                        

...

<IPython.core.display.Javascript object>

[jovian] Updating notebook "b00034290bee47dab68436fc6b62fbd8" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/harangdev/b00034290bee47dab68436fc6b62fbd8
FOLD: 3
Epoch 1/1000
Score: 0.6427 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1586
3021/3021 [==============================] - 1585s 525ms/step - loss: 0.1957 - multiply_loss: 0.1957 - cls_output_loss: 0.3659
Epoch 2/1000
Score: 0.6376 | LR to: 0.0003000000142492354                                                                                                     
Restoring Best Weights
Reducing LR on Plateau, Setting LR to 6.000000284984708e-05
TIME SPENT: 1537
3021/3021 [==============================] - 1537s 509ms/step - loss: 0.1741 - multiply_loss: 0.1741 - cls_output_loss: 0.3128
Epoch 3/1000
Score: 0.6460 | LR to: 6.000000212225132e-05     

<IPython.core.display.Javascript object>

[jovian] Updating notebook "b00034290bee47dab68436fc6b62fbd8" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/harangdev/b00034290bee47dab68436fc6b62fbd8
FOLD: 4
Epoch 1/1000
Score: 0.6307 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1592
3021/3021 [==============================] - 1591s 527ms/step - loss: 0.1948 - multiply_loss: 0.1948 - cls_output_loss: 0.3659
Epoch 2/1000
Score: 0.6360 | LR to: 0.0003000000142492354                                                                                                     
TIME SPENT: 1532
3021/3021 [==============================] - 1533s 507ms/step - loss: 0.1738 - multiply_loss: 0.1738 - cls_output_loss: 0.3069
Epoch 3/1000
Score: 0.6390 | LR to: 0.0003000000142492354                                                                                        

KeyboardInterrupt: 

In [9]:
print(cb.best_score)
print(cb.lr_schedule)       

save_name = f'{arch_name}_{i}_{str(cb.best_score)[:6]}'
model.save(f'{stage}/models/seg/{save_name}.h5')

val_pred = np.zeros(y_val.shape, dtype=np.float32)
tmp_test_pred = np.zeros((len(test),*img_size,4), dtype=np.float32)
for hf in [{'p': 1.0}, False]:
    for vf in [{'p': 1.0}, False]:
        for case, data in [('x_val', x_val), ('test', test)]:
            tta_generator = vision.Generator(
                data,
                batch_size=batch_size*4,
                augment='image',
                hf=hf,
                vf=vf,
                preprocess_input=preprocess_input
            )
            single_pred = model.predict_generator(tta_generator, verbose=1)[0]
            if hf:
                single_pred = np.flip(single_pred, axis=2)
            if vf:
                single_pred = np.flip(single_pred, axis=1)
            if case == 'x_val':
                val_pred += single_pred/4
            elif case == 'test':
                tmp_test_pred += single_pred/4


utils.to_pickle(f'{stage}/oof_preds/seg/{save_name}.pkl', val_pred)
utils.to_pickle(f'{stage}/test_preds/seg/{save_name}.pkl', tmp_test_pred)

oof_pred[fold[1]] = val_pred
test_pred += tmp_test_pred/5

score_str = str(dice_metric(y_data, oof_pred>0.5))[:6]
print(f'OOF SCORE: {score_str}')
utils.to_pickle(f'{stage}/oof_preds/seg/{arch_name}_{score_str}.pkl', oof_pred)
utils.to_pickle(f'{stage}/test_preds/seg/{arch_name}_{score_str}.pkl', test_pred)

jovian.commit(nb_filename='segmentation.ipynb', secret=True, env_type='pip')

0.6408411423397271
[0.0003, 0.0003, 0.0003, 1.2e-05]
155/155 [==============================] - 61s 396ms/step
OOF SCORE: 0.6482
[jovian] Saving notebook..


<IPython.core.display.Javascript object>

[jovian] Updating notebook "b00034290bee47dab68436fc6b62fbd8" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/harangdev/b00034290bee47dab68436fc6b62fbd8


---

## STAGE 1

### EfficientNetB2: 0.6407

* 0.6448
* 0.6315
* 0.6409
* 0.6377
* 0.6353

### EfficientNetB3: 0.6444

* 0.6456
* 0.6437
* 0.6487
* 0.6440
* 0.6394

### EfficientNetB4: 0.6422

* 0.6418
* 0.6384
* 0.6452
* 0.6377
* 0.6375

### Ensemble: 0.6473 / 0.6553(post)

## STAGE 2

### EfficientNetB2: 0.6492

* 0.6520
* 0.6461
* 0.6592
* 0.6492
* 0.6426

### EfficientNetB3: 0.6514

* 0.6516
* 0.6484
* 0.6627
* 0.6504
* 0.6474

### FPN EfficientNetB1: 0.6482

* 0.6503
* 0.6489
* 0.6568
* 0.6509
* 0.6408

### Ensemble: 0.6 / 0.6(post)